In [ ]:
import os
import loompy as lp
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
def create_loom(adata, path):
    row = {"Gene": np.array(adata.var_names)}
    col = {"CellID": np.array(adata.obs_names),"nGene": np.array(np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,"nUMI": np.array(np.sum(adata.X.transpose() , axis=0)).flatten()}
    matrix = adata.X.transpose()
    lp.create(path, matrix, row, col)
    
adata.X = adata.layers['counts']
create_loom(adata, '../../output/annotated.loom')